In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import requests

In [75]:
url = ('https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&facets[respondent][]=NY&facets[type][]=D&start=2019-01-01T00&end=2024-08-01T00&sort[0][column]=period&sort[0][direction]=desc&api_key=') + "oC2FREcg0Z5k0jkM8Bv3XOXtBfk1F9aYmDo2ZKLL"

In [76]:
data = requests.get(url).json()['response']['data']

print(data)

[{'period': '2024-08-01T00', 'respondent': 'NY', 'respondent-name': 'New York', 'type': 'D', 'type-name': 'Demand', 'value': '27194', 'value-units': 'megawatthours'}, {'period': '2024-07-31T23', 'respondent': 'NY', 'respondent-name': 'New York', 'type': 'D', 'type-name': 'Demand', 'value': '27566', 'value-units': 'megawatthours'}, {'period': '2024-07-31T22', 'respondent': 'NY', 'respondent-name': 'New York', 'type': 'D', 'type-name': 'Demand', 'value': '27552', 'value-units': 'megawatthours'}, {'period': '2024-07-31T21', 'respondent': 'NY', 'respondent-name': 'New York', 'type': 'D', 'type-name': 'Demand', 'value': '27069', 'value-units': 'megawatthours'}, {'period': '2024-07-31T20', 'respondent': 'NY', 'respondent-name': 'New York', 'type': 'D', 'type-name': 'Demand', 'value': '26673', 'value-units': 'megawatthours'}, {'period': '2024-07-31T19', 'respondent': 'NY', 'respondent-name': 'New York', 'type': 'D', 'type-name': 'Demand', 'value': '26238', 'value-units': 'megawatthours'}, {'p

In [77]:
data = pd.DataFrame(data)
print(data.shape)

(5000, 7)


In [68]:
print(data)

             period respondent respondent-name type type-name  value  \
0     2024-08-01T00         NY        New York    D    Demand  27194   
1     2024-07-31T23         NY        New York    D    Demand  27566   
2     2024-07-31T22         NY        New York    D    Demand  27552   
3     2024-07-31T21         NY        New York    D    Demand  27069   
4     2024-07-31T20         NY        New York    D    Demand  26673   
...             ...        ...             ...  ...       ...    ...   
4995  2024-01-05T21         NY        New York    D    Demand  19249   
4996  2024-01-05T20         NY        New York    D    Demand  18874   
4997  2024-01-05T19         NY        New York    D    Demand  18780   
4998  2024-01-05T18         NY        New York    D    Demand  18779   
4999  2024-01-05T17         NY        New York    D    Demand  18991   

        value-units  
0     megawatthours  
1     megawatthours  
2     megawatthours  
3     megawatthours  
4     megawatthours  
...

In [48]:
demand_daily = data[['period', 'value']].rename(columns={'period': 'date', 'value': 'demand'})
demand_daily['date'] = pd.to_datetime(demand_daily['date'], infer_datetime_format=True)
print(demand_daily)

TypeError: unhashable type: 'list'

In [46]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.7143,
	"longitude": -74.006,
	"start_date": "2019-01-01",
	"end_date": "2024-08-01",
	"hourly": "temperature_2m",
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe) #48948 hours

Coordinates 40.738136291503906°N -74.04254150390625°E
Elevation 51.0 m asl
Timezone b'America/New_York' b'EDT'
Timezone difference to GMT+0 -14400 s
                           date  temperature_2m
0     2019-01-01 04:00:00+00:00        7.872500
1     2019-01-01 05:00:00+00:00        8.672500
2     2019-01-01 06:00:00+00:00        9.172500
3     2019-01-01 07:00:00+00:00       10.822500
4     2019-01-01 08:00:00+00:00       12.572500
...                         ...             ...
48955 2024-08-01 23:00:00+00:00       29.572498
48956 2024-08-02 00:00:00+00:00       27.772499
48957 2024-08-02 01:00:00+00:00       26.222500
48958 2024-08-02 02:00:00+00:00       25.472500
48959 2024-08-02 03:00:00+00:00       25.672499

[48960 rows x 2 columns]
